In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWgypaQTY5c7BZqlXZ9Xa0BL2GbOy6uezk6inlpl3XAMauDKxrx0E8U

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
import geemap
import geopandas

In [4]:
## specify areas of interest / districts
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#Districts = ee.FeatureCollection('users/emackres/Wards/Kigali_Sectors_2012_PrincipleComponents')
#Districts = ee.FeatureCollection('users/emackres/KingCountyJurisdictions')
#UrbanShiftCities = ee.FeatureCollection('users/emackres/UrbanShift_cities')
Districts= UrbanShiftCities.filter(ee.Filter.inList('id',['CRI-San_Jose'])) 
#Districts = ee.FeatureCollection('users/emackres/Wards/San_Jose_municipalities')

#print(Districts.limit(1).getInfo())

NameError: name 'UrbanShiftCities' is not defined

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=10)

In [ ]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [ ]:
Map.addLayer(Districts,{},"Districts")

In [ ]:
# get bounding box for Districts area

bb = Districts.union().geometry().bounds(1000)
print(bb.getInfo())
#Map.addLayer(bb,{},"Bounding box")

# get N, S, E, W coordinates from bounding box
# from https://gis.stackexchange.com/questions/318959/get-lon-lat-of-a-top-left-corner-for-geometry-in-google-earth-engine

## return the list of coordinates
listCoords = ee.Array.cat(bb.coordinates(), 1); 
##Casting it to an array makes it possible to slice out the x and y coordinates:
##get the X-coordinates
xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());

## Reducing each array reveals then the min and max values:
## reduce the arrays to find the max (or min) value
West = ee.Number(xCoords.reduce('min', [0]).get([0,0])).getInfo(); print('West',West);
East = ee.Number(xCoords.reduce('max', [0]).get([0,0])).getInfo(); print('East',East);
North = ee.Number(yCoords.reduce('min', [0]).get([0,0])).getInfo(); print('North',North);
South = ee.Number(yCoords.reduce('max', [0]).get([0,0])).getInfo(); print('South',South);

In [ ]:
# use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections

tags = {'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #{'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} #}
RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
#RecSites = geemap.osm_gdf_from_bbox(9.09932636176124, 8.833260675723484, 38.90627204838861, 38.63876608591662, {'leisure':'park'}).drop(columns=['contact:p.o.box'])
RecSites = RecSites.reset_index()
print(RecSites.shape[0])
print(RecSites.crs)
RecSites.sort_index()#.sample(3)
#.sort_values(by=['osmid'],axis='columns')

In [ ]:
## keep only columns desired to reduce file size 
RecSites = RecSites.loc[:, ['osmid','geometry']] #['element_type','osmid','leisure','boundary','name','geometry']
RecSites#.sample(3)

In [ ]:
##Optional and not working in useful way - use only if "EEException" occurs below and willing to sacrafice geographic precision.
# Simplify geography of geodataframe - 

RecSitesGeo = RecSites.geometry.simplify(0.00001) #parameter is tolerance for error in units of the CRS - degrees for default EPSG:4326
RecSitesSimp = RecSites.set_geometry(RecSitesGeo)
RecSitesSimp#.sample(3)

In [ ]:
# convert Geodataframe to GeoJson
RecSitesGJ = geemap.gdf_to_geojson(RecSites) #RecSitesSimp

In [ ]:
# convert GeoJson to ee.FeatureCollection
RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])
#print(RecSitesFC.first().toDictionary().getInfo())


In [ ]:
# add recreation sites to map 
Map.addLayer(RecSitesFC, {}, 'Recreation Sites')
#Map.centerObject(RecSitesFC, 11)
Map

# Large number of columns or complicated features may lead to "EEException: Request payload size exceeds the limit:" errors when trying to add to map or do additional operations. 
# Reduce number of columns in geodataframe to address. Secondarily, simplifying the geometries of each feature could be attempted. Use the lowest tolerance parameter possible. 

In [ ]:
## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.

WCprojection = WorldCover.projection(); 
#RecSitesImg = RecSitesFC.filter(ee.Filter.eq('element_type','way')).reduceToImage(properties=['osmid'],reducer=ee.Reducer.first());

RecSitesImg = RecSitesFCsimp.style(
  color='gray',
).reproject(
      crs= WCprojection
    )
#print(RecSitesImg.getInfo())

## Display image.
#Map.addLayer(RecSitesImg, {},"Rec Sites Image");

In [ ]:
# create image with two bands: BuiltupPixels and UrbanOpenPixels
Builtup = WorldCover.updateMask(WorldCover.eq(50)).rename("BuiltupPixels")
UrbanOpen = RecSitesImg.updateMask(WorldCover.eq(50)).select(1).rename("UrbanOpenPixels")

#Map.addLayer(Builtup, {},"Builtup");
#Map.addLayer(UrbanOpen, {},"UrbanOpen");

comb = Builtup.addBands([UrbanOpen])
#print(comb.getInfo)

In [ ]:
## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature

OpenBuiltcount=comb.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

#print(OpenBuiltcount.limit(1).getInfo())

In [ ]:
# Function to translate pixel counts into area and percents

def CountToArea(feat):
    feat = ee.Feature(feat)
    area = ee.Number(feat.area()).multiply(0.000001)
    UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))

    return feat.set({
        'TotalareaKM2': area,
        'UrbanOpenAreaKM2': UrbanOpenArea,
        'BuiltupAreaKM2': BuiltupArea,
        'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea
    })

# apply function to FeatureCollection
OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea)
#print(OpenBuiltAreaPct.limit(1).getInfo())

In [ ]:
Tpctfills = ee.Image().paint(**{'featureCollection': OpenBuiltAreaPct,'color': 'OpenAreaPctofBuiltUpArea'})

fillspalette = ['red', 'green']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':0.1}, 'Open Area as % of built-up area', True, 0.65)
Map

In [ ]:
# Download OpenBuiltAreaPct as shapefile through URL

# function for converting GeometryCollection to Polygon/MultiPolygon
def filter_polygons(ftr):
    geometries = ftr.geometry().geometries()
    geometries = geometries.map(
        lambda geo: ee.Feature(ee.Geometry(geo)).set('geoType', ee.Geometry(geo).type())
    )

    polygons = (
        ee.FeatureCollection(geometries)
        .filter(ee.Filter.eq('geoType', 'Polygon'))
        .geometry()
    )
    return ee.Feature(polygons).copyProperties(ftr)

OpenBuiltAreaPctSHP = OpenBuiltAreaPct.map(filter_polygons)

col_names = OpenBuiltAreaPctSHP.first().propertyNames().getInfo()
print("Column names: ", col_names)

url = OpenBuiltAreaPctSHP.getDownloadURL("shp", col_names, 'OpenBuiltAreaPctSHP')
print(url)


In [ ]:
# Download FeatureCollection of OpenBuiltAreaPct as shapefile to Google Drive

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'SHP',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}
#print('Exporting {}'.format(OpenBuiltAreaPct))
task = ee.batch.Export.table.toDrive(OpenBuiltAreaPct, 'OpenBuiltAreaPct', **task_config)
task.start()

In [ ]:

# Download FeatureCollection of OSM data (ways/polygons only) to Google Drive as geojson
# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'fileFormat': 'Geojson',
    #'selectors': col_names,  # a list of properties/attributes to be exported
}

#print('Exporting {}'.format(RecSitesFC))
task = ee.batch.Export.table.toDrive(RecSitesFC, 'RecSites', **task_config)
task.start()